<a href="https://colab.research.google.com/github/m-bashari-m/sample-DNN-projects/blob/main/4.3_cat_vs_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [269]:
import tensorflow as tf
import os

_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir =  "/root/.keras/datasets/cats_and_dogs_filtered/train"
validation_dir = "/root/.keras/datasets/cats_and_dogs_filtered/validation"

In [270]:
train_ds = tf.data.Dataset.list_files(str(train_dir+'/*/*'))
val_ds = tf.data.Dataset.list_files(str(validation_dir+'/*/*'))

In [271]:
for path in train_ds.take(5):
  print(path)

tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/cats/cat.299.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/cats/cat.747.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.222.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.697.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.341.jpg', shape=(), dtype=string)


In [272]:
def prepare_labeled_data(path):
  class_names = tf.constant(['cat', 'dog'])

  categorical_label = tf.strings.split(path, '/')[-1]
  categorical_label = tf.strings.split(categorical_label, '.')[0]
  numerical_label = tf.cast(categorical_label == class_names[0], tf.int8) # cat=1, dog=0

  image = tf.image.decode_image( tf.io.read_file(path), expand_animations=False )
  
  return image,  numerical_label

In [273]:
def augmentation(img):
  img = tf.image.resize_with_crop_or_pad(img, 260, 260)
  img = tf.image.random_crop(img, size=[256, 256, 3])
  img = tf.image.random_brightness(img, max_delta=.2)
  img = tf.image.random_flip_left_right(img)
  return img

In [274]:
def normalization(img):
  img = tf.image.convert_image_dtype(img, tf.float32)
  return img / 255

In [275]:
def train_preprocessing(path):
  img, label = prepare_labeled_data(path)
  img = augmentation(img)
  img = normalization(img)
  return img, label

In [276]:
def test_preprocessing(path):
  img, label = prepare_labeled_data(path)
  img = normalization(img)
  img = tf.image.resize(img, [256, 256])
  return img, label

In [277]:
BATCH_SIZE=64
AUTOTUNE = tf.data.AUTOTUNE
BUFFER_SIZE=1000

In [278]:
train_ds = train_ds.map(train_preprocessing).shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
val_ds = val_ds.map(test_preprocessing).batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [279]:
for img, lbl in train_ds.take(3):
  print(img.shape)

(64, 256, 256, 3)
(64, 256, 256, 3)
(64, 256, 256, 3)


In [280]:
class NormalizationAndActivation(tf.keras.layers.Layer):
  def __init__(self, activation='relu', **kwargs):
    super(NormalizationAndActivation, self).__init__(**kwargs)
    self.activation = tf.keras.activations.get(activation)
    self.batch_norm = tf.keras.layers.BatchNormalization()

  def call(self, X):
    return self.batch_norm(self.activation(X))


In [281]:
class CatVsDog(tf.keras.models.Model):
  def __init__(self, num_classes=2, activation='relu', **kwargs):
    super(CatVsDog, self).__init__(**kwargs)

    self.model_layers = [
                   tf.keras.layers.Conv2D(64, (3,3), strides=(2,2), padding='same', name='conv_1_1'),
                   NormalizationAndActivation(name='bn_1_2'),
                   tf.keras.layers.Conv2D(64, (3,3), strides=(2,2), padding='same', name='conv_2_3'),
                   NormalizationAndActivation(name='bn_2_4'),
                   tf.keras.layers.MaxPooling2D(2, name='max_pool_1_5'),

                   tf.keras.layers.Conv2D(128, (3,3), strides=(2,2), padding='same', name='conv_3_6'),
                   NormalizationAndActivation(name='bn_3_7'),
                   tf.keras.layers.Conv2D(128, (3,3), strides=(2,2), padding='same', name='conv_4_8'),
                   NormalizationAndActivation(name='bn_4_9'),
                   tf.keras.layers.MaxPooling2D(2, name='max_pool_2_10'),
                   
                   tf.keras.layers.Conv2D(512, (3,3), strides=(2,2), padding='same', name='conv_5_11'),
                   NormalizationAndActivation(name='bn_5_12'),
                   tf.keras.layers.Conv2D(512, (3,3), strides=(2,2), padding='same', name='conv_6_13'),
                   NormalizationAndActivation(name='bn_6_14'),
                  #  tf.keras.layers.MaxPooling2D(2, name='max_pool_3_15'),
                   tf.keras.layers.Dropout(.2),

                   tf.keras.layers.Conv2D(1024, (3,3), strides=(2,2), padding='same', name='conv_7_16'),
                   NormalizationAndActivation(name='bn_7_17'),
                   tf.keras.layers.Conv2D(1024, (3,3), strides=(2,2), padding='same', name='conv_8_18'),
                   NormalizationAndActivation(name='bn_8_19'),
                  #  tf.keras.layers.MaxPooling2D(2, name='max_pool_4_20'),
                   tf.keras.layers.Dropout(.2),

                   tf.keras.layers.Conv2D(2048, (3,3), strides=(2,2), padding='same', name='conv_9_21'),
                   NormalizationAndActivation(name='bn_9_22'),
                   tf.keras.layers.Conv2D(2048, (3,3), strides=(2,2), padding='same', name='conv_10_23'),
                   NormalizationAndActivation(name='bn_10_24'),
                   tf.keras.layers.MaxPooling2D(2, name='max_pool_5_25'),
                   tf.keras.layers.Dropout(.2),
                   
                   tf.keras.layers.GlobalAveragePooling2D(name='avg_pool_1_26'),
                   tf.keras.layers.Flatten(),
                   tf.keras.layers.Dense(200, name='dense_1_27'),
                   NormalizationAndActivation(name='bn_11_28'),
                   tf.keras.layers.Dropout(.5),
                   tf.keras.layers.Dense(num_classes, name='classifier')]
  

  def call(self, X):
    for layer in self.model_layers:
      X = layer(X)

    return X

In [282]:
from tensorflow.python.framework.tensor_conversion_registry import get
class CatVsDog(tf.keras.models.Model):
  def __init__(self, num_classes=2, activation='relu', **kwargs):
    super(CatVsDog, self).__init__(**kwargs)

    self.model_layers = [
                   tf.keras.layers.Conv2D(64, (3,3), padding='same', name='conv_1_1'),
                   tf.keras.layers.Activation(activation),
                   tf.keras.layers.Conv2D(64, (3,3), padding='same', name='conv_2_3'),
                   NormalizationAndActivation(name='bn_2_4'),
                   tf.keras.layers.MaxPooling2D(2, name='max_pool_1_5'),

                   tf.keras.layers.Conv2D(128, (3,3), padding='same', name='conv_3_6'),
                   tf.keras.layers.Activation(activation),
                   tf.keras.layers.Conv2D(128, (3,3), padding='same', name='conv_4_8'),
                   NormalizationAndActivation(name='bn_4_9'),
                   tf.keras.layers.MaxPooling2D(2, name='max_pool_2_10'),
                   
                   tf.keras.layers.Conv2D(512, (3,3), padding='same', name='conv_5_11'),
                   tf.keras.layers.Activation(activation),
                   tf.keras.layers.Conv2D(512, (3,3), padding='same', name='conv_6_13'),
                   NormalizationAndActivation(name='bn_6_14'),
                   tf.keras.layers.MaxPooling2D(2, name='max_pool_3_15'),
                   tf.keras.layers.Dropout(.2),

                   tf.keras.layers.Conv2D(1024, (3,3), padding='same', name='conv_7_16'),
                   tf.keras.layers.Activation(activation),
                   tf.keras.layers.Conv2D(1024, (3,3), padding='same', name='conv_8_18'),
                   NormalizationAndActivation(name='bn_8_19'),
                   tf.keras.layers.MaxPooling2D(2, name='max_pool_4_20'),
                   tf.keras.layers.Dropout(.2),
                   
                   tf.keras.layers.GlobalAveragePooling2D(name='avg_pool_1_26'),
                   tf.keras.layers.Flatten(),
                   tf.keras.layers.Dense(200, name='dense_1_27'),
                   NormalizationAndActivation(name='bn_11_28'),
                   tf.keras.layers.Dropout(.5),
                   tf.keras.layers.Dense(num_classes, name='classifier')]
  

  def call(self, X):
    for layer in self.model_layers:
      X = layer(X)

    return X

In [283]:
import os

path = '/content/checkpoints'
try:
    os.mkdir(path)
except OSError as error:
    pass

checkpoint = tf.keras.callbacks.ModelCheckpoint(path, save_wights_only=True)


In [284]:
model = CatVsDog()

model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Nadam(),
              metrics=['accuracy'])

In [ ]:
model.fit(train_ds, 
          epochs=25,
          batch_size=128,
          validation_data=val_ds,
          callbacks=[checkpoint])

Epoch 1/25
20/32 [=================>............] - ETA: 1:18 - loss: 6.5103 - accuracy: 0.5117

In [ ]:
tf.keras.utils.plot_model(model, 'model1.png', show_shapes=True)